In [19]:
from svgpathtools import Path, Line, QuadraticBezier, CubicBezier, Arc, disvg, wsvg
import numpy as np

seg1 = QuadraticBezier(100+100j, 200+500j, 300+100j)  # A cubic beginning at (300, 100) and ending at (200, 300)
seg2 = QuadraticBezier(400+150j, 0+100j, 400+200j)  # A line beginning at (200, 300) and ending at (250, 350)
seg3 = Line(50+100j,400+200j)
#Four Intersections
#seg1 = QuadraticBezier(250+100j, 200+500j, 300+100j)  # A cubic beginning at (300, 100) and ending at (200, 300)
#seg2 = QuadraticBezier(400+150j, 0+100j, 400+200j)  # A line beginning at (200, 300) and ending at (250, 350)
path = Path(seg1, seg2, seg3)

off1 = QuadraticBezier(100+100j, 200+500j, 300+100j)  # A cubic beginning at (300, 100) and ending at (200, 300)
off2 = QuadraticBezier(400+150j, 0+100j, 400+200j)  # A line beginning at (200, 300) and ending at (250, 350)
off3 = Line(50+100j,400+200j)

off_path = Path(off1,off2,off3)

In [4]:
import time

def mesure(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        execution_time = end_time - start_time
        print(f"'{func.__name__}': {execution_time:.6f} seconds.")
        return result
    return wrapper

In [68]:
@mesure
def offset_segment(seg,distance,res=5) -> list: 
    distance = distance * - 1

    if isinstance(seg,Line):
        offset_vector = distance * seg.normal(0)
        offset_seg = Line(seg.point(0) + offset_vector,seg.point(1) + offset_vector)
        return offset_seg

    elif isinstance(seg,(Arc, QuadraticBezier, Line)):

        offset_seg = Path()
        curr = seg.point(0) + distance * seg.normal(0)
        curr_tan = seg.unit_tangent(0)
        for k in range(1,res+1):
            t=k / res
            next = seg.point(t) + distance * seg.normal(t)
            next_tan = seg.unit_tangent(t)
            
            # Having curr/next as start/end  of each segment,
            # vectors curr_tan/next_tan pointing to a common point
            # can be used as refference for offset segment as bezier curve
            if curr_tan.real == 0:
                X = curr.real
                Y = (next_tan.imag/next_tan.real) * (X - next.real) + next.imag
            
            elif next_tan.real == 0:
                X = next.real
                Y = (curr_tan.imag/curr_tan.real) * (X - curr.real) + curr.imag
            
            else:
                Ax = curr.real; Ay = curr.imag
                Bx = next.real; By = next.imag

                ma = curr_tan.imag/curr_tan.real
                mb = next_tan.imag/next_tan.real

                # solving for lines intersection
                X = ((By-Ay) - (mb * Bx - ma * Ax))/(ma-mb)
                Y = (ma*mb * (Bx-Ax) - (By*ma-Ay*mb))/(mb-ma)
            
            control = complex(X,Y)
            new_element = QuadraticBezier(curr,control,next)
            # if offset_seg: print(new_element, offset_seg[-1])
            
            # print(offset_seg.intersect(new_element))
            # Before appending new_element, check if it intersects with some
            # non consecutive previous element, if so, delete all elements between
            # both of them, chop tails and check for continuity.
            offset_seg.append(new_element)
            curr = next
            curr_tan = next_tan
        
        return offset_seg  #-> Path

    else:
        raise TypeError("Input `segment` should be a Line, "
                        "QuadraticBezier, CubicBezier, or Arc object.")


def remove_loops(path:Path) -> Path:
    for i, seg in enumerate(path[:-2]):
        seg3 = [path[i+2]]
        print(seg,seg3)


In [53]:
import matplotlib.pyplot as plt
from svgpathtools import Line, QuadraticBezier, CubicBezier, Arc, Path
from svgpathtools import svg2paths, svg2paths2, wsvg
Path.iscontained = False

@mesure
def char2paths(char): # Implement character with multiple path, maybe ussing parsing

    plt.figure(figsize=(4, 4))
    plt.text(0.5, 0.5, char[0], fontsize=200, ha='center', va='center')
    plt.axis('off')
    filename='.matplot'
    plt.savefig(filename, format='svg')
    plt.close()
    
    paths, _, _ = svg2paths2(filename)
    paths = paths[1].continuous_subpaths()
    wsvg(paths, filename='svgtools.svg')
    
    return paths #list

@mesure
def set_contained_attr(paths: list) -> None:
    for path in paths:
        path.iscontained = any(path.is_contained_by(compare) for compare in paths if compare != path)

In [69]:
test = offset_segment(off2,-10)
remove_loops(test)
nodes = [seg.start for seg in test] + [test.end]
wsvg([off2,test],filename='display_c.svg',nodes=nodes)

'offset_segment': 0.000000 seconds.
QuadraticBezier(start=(398.7596526541079+159.92277876713666j), control=(318.96415912689054+149.94834207623447j), end=(271.16954520146265+145.96545758244883j)) [QuadraticBezier(start=(209.24034734589208+143.92277876713666j), control=(226.01305931659536+141.82618977079875j), end=(212.47213595499957+135.05572809000085j))]
QuadraticBezier(start=(271.16954520146265+145.96545758244883j), control=(224.2071681746219+142.05192616354543j), end=(209.24034734589208+143.92277876713666j)) [QuadraticBezier(start=(212.47213595499957+135.05572809000085j), control=(227.66563145999496+142.65247584249852j), end=(274.80000000000007+156.40000000000003j))]
QuadraticBezier(start=(209.24034734589208+143.92277876713666j), control=(226.01305931659536+141.82618977079875j), end=(212.47213595499957+135.05572809000085j)) [QuadraticBezier(start=(274.80000000000007+156.40000000000003j), control=(322.6136624629399+170.34565155169085j), end=(402.42535625036334+190.29857499854668j))]
